In [12]:
import pandas as pd
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn
import os

# === 1. 路径设置 ===
base_path = r"C:\Users\Lenovo\Desktop\大四上\人工智能与Python程序设计\HW2"
excel_path = os.path.join(base_path, "att2_Uni_Major_List.xlsx")
template_path = os.path.join(base_path, "att1_Application_Template.docx")
output_dir = os.path.join(base_path, "output_docs")
os.makedirs(output_dir, exist_ok=True)

# === 2. 读取Excel的两个sheet ===
univ_df = pd.read_excel(excel_path, sheet_name="univ")
field_df = pd.read_excel(excel_path, sheet_name="field")

# === 3. 定义替换函数 ===
def replace_placeholders(doc, replacements):
    """
    在docx文档中用replacements字典替换形如 [key] 的字段
    """
    italic_keys = ["grad_univ", "app_col", "journal"]
    for p in doc.paragraphs:
        for key, value in replacements.items():
            p.text = p.text.replace(f"[{key}]", str(value))
         # 设置字体
        for run in p.runs:
            run.font.name = 'Times New Roman'
            run._element.rPr.rFonts.set(qn('w:eastAsia'), 'Times New Roman')
            run.font.size = Pt(12)  # 可选，设置字号为12
    return doc

# === 4. 遍历每个申请者，生成对应Word文档 ===
for i, row in univ_df.iterrows():
    replacements = {
        "applicant": row.get("applicant", ""),
        "grad_univ": row.get("grad_univ", ""),
        "program": row.get("program", ""),
        "app_col": row.get("app_col", "")
    }

    # 如果你还要补充field表中的信息，可以这样添加：
    for idx, row_field in field_df.iterrows():
        for col in ["field", "journal", "skill"]:  # 遍历需要添加的列
            if col in field_df.columns:
                replacements[col] = row_field[col]  # 用这一行的值填充
        
        # 打开模板
        doc = Document(template_path)
        
        # 替换方括号内容
        doc = replace_placeholders(doc, replacements)
    
        # 保存输出
        output_filename = f"{i*3+idx+1}_{row['applicant']}_Application.docx"
        output_path = os.path.join(output_dir, output_filename)
        doc.save(output_path)

print("✅ 所有文档已生成，保存在:", output_dir)


✅ 所有文档已生成，保存在: C:\Users\Lenovo\Desktop\大四上\人工智能与Python程序设计\HW2\output_docs
